In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import numpy as np
import os

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [2]:
from torchvision.transforms import v2
# Define data transformations
transform_train = v2.Compose([
    v2.Grayscale(num_output_channels=3),  # Convert grayscale images to 3-channel images
    v2.Resize((224, 224)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(degrees=(0,180)),
    v2.ToTensor(),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale images to 3-channel images
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
data_path = ''  # Replace with the actual path
train_dataset = datasets.ImageFolder(os.path.join(data_path, 'train'), transform=transform_train)
test_dataset = datasets.ImageFolder(os.path.join(data_path, 'test'), transform=transform_test)


# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f'Training samples: {len(train_loader.dataset)}')
print(f'Test samples: {len(test_loader.dataset)}')


Training samples: 24176
Test samples: 6043


C:\Users\awael\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
model_f= models.resnet34(weights=True)
num_ftrs = model_f.fc.in_features
num_ftrs

C:\Users\awael\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


512

In [4]:
model_f.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs,512),
    nn.ReLU(),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,5)
    )

model_f = model_f.to(device)

In [5]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_f.parameters(), lr=0.00005)

In [6]:

# Early stopping parameters
early_stopping_patience = 30
best_test_loss = np.inf
best_test_acc = 0
early_stopping_counter = 0

In [7]:
# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

# Validation function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(test_loader.dataset)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc



In [8]:
num_epochs = 100
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
    train_loss, train_acc = train(model_f, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model_f, test_loader, criterion, device)
    
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
    
    # Check if testidation loss has improved
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        early_stopping_counter = 0
        torch.save(model_f.state_dict(), 'res34_res34.pth')  # Save the best model_2
    else:
        early_stopping_counter += 1
    
    # Early stopping
    if early_stopping_counter >= early_stopping_patience:
        print("Early stopping triggered")
        break

Epoch 1/100


Train Loss: 1.1753, Train Acc: 50.82%
Test Loss: 1.0386, Test Acc: 58.10%
Epoch 2/100


Train Loss: 0.8806, Train Acc: 65.74%
Test Loss: 0.8754, Test Acc: 65.12%
Epoch 3/100


Train Loss: 0.7952, Train Acc: 69.12%
Test Loss: 0.8752, Test Acc: 65.96%
Epoch 4/100


Train Loss: 0.7524, Train Acc: 71.20%
Test Loss: 0.8135, Test Acc: 68.51%
Epoch 5/100


...

Train Loss: 0.0776, Train Acc: 97.27%
Test Loss: 1.1916, Test Acc: 73.59%
Epoch 97/100


Train Loss: 0.0766, Train Acc: 97.37%
Test Loss: 1.2064, Test Acc: 72.66%
Epoch 98/100


Train Loss: 0.0766, Train Acc: 97.23%
Test Loss: 1.2199, Test Acc: 72.99%
Epoch 99/100


Train Loss: 0.0751, Train Acc: 97.35%
Test Loss: 1.2669, Test Acc: 72.79%
Epoch 100/100


KeyboardInterrupt: 

In [12]:
from PIL import Image
import torch
from torchvision import transforms

# Function to load and preprocess the image
def load_image(image_path):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),  # Convert grayscale images to 3-channel images
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB') 
    image = transform(image)
    image = image.unsqueeze(0)  # Add a batch dimension
    return image.to(device)


In [13]:
# Function to predict the emotion from an image
def predict_emotion(image_path, model_f, class_names):
    # Load and preprocess the image
    image = load_image(image_path)
    
    # Set the model to evaluation mode
    model_f.eval()
    
    with torch.no_grad():
        # Make predictions
        outputs = model_f(image)
        _, predicted = torch.max(outputs, 1)
    
    # Get the predicted class name
    predicted_class = class_names[predicted.item()]
    return predicted_class


In [30]:
# Assuming you have the model loaded and trained
model_f.load_state_dict(torch.load('res34_res34.pth'))  # Load the best model
class_names = train_loader.dataset.classes

# Path to the image you want to predict
image_path = 'my6.jpg'  # Replace with the actual path

# Predict the emotion
predicted_emotion = predict_emotion(image_path, model_f, class_names)
print(f'The predicted emotion is: {predicted_emotion}')


C:\Users\awael\AppData\Local\Temp\ipykernel_18652\2864598909.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_f.load_state_dict(torch.load('res34_res34.pth'))  # Lo

The predicted emotion is: angry


In [28]:
class_names = train_loader.dataset.classes
class_names

['angry', 'happy', 'neutral', 'sad', 'surprised']